In [1]:
import akshare as ak

In [6]:
y2601_df = ak.futures_main_sina(symbol='Y2601')

In [6]:
# 1. 获取主力连续合约数据（以螺纹钢为例，symbol通过ak.futures_display_main_sina()确认）
import pandas as pd


symbol = "RB0"  # 螺纹钢主力连续
main_df = ak.futures_main_sina(symbol=symbol, start_date="20180101", end_date="20231231")
print(main_df.head())
# main_df["日期"] = pd.to_datetime(main_df["日期"])
# main_df = main_df.sort_values("日期").set_index("日期")

# # 2. 定位换月点（价格跳空超过阈值的日期，阈值根据品种特性调整）
# main_close = main_df["收盘价"]
# # 计算单日涨跌幅绝对值（跳空幅度）
# price_jump = main_close.pct_change().abs()
# # 筛选跳空超过5%的日期作为候选换月点（螺纹钢通常波动较小，5%可有效识别换月）
# candidate_dates = price_jump[price_jump > 0.05].index.tolist()
# print(f"候选换月点：{candidate_dates}")

           日期   开盘价   最高价   最低价   收盘价      成交量      持仓量  动态结算价
0  2018-01-02  3801  3886  3794  3874  2961436  2413572   3842
1  2018-01-03  3866  3877  3803  3809  3523844  2433636   3845
2  2018-01-04  3805  3842  3788  3819  3747764  2581090   3811
3  2018-01-05  3812  3827  3760  3798  3941160  2744436   3791
4  2018-01-08  3795  3820  3733  3818  4329290  2717212   3772
候选换月点：[Timestamp('2018-03-23 00:00:00'), Timestamp('2018-03-28 00:00:00'), Timestamp('2018-11-28 00:00:00'), Timestamp('2019-04-01 00:00:00'), Timestamp('2019-08-26 00:00:00'), Timestamp('2020-02-03 00:00:00'), Timestamp('2021-05-06 00:00:00'), Timestamp('2021-05-10 00:00:00'), Timestamp('2021-05-19 00:00:00'), Timestamp('2021-05-26 00:00:00'), Timestamp('2021-08-02 00:00:00'), Timestamp('2021-10-21 00:00:00'), Timestamp('2021-10-27 00:00:00'), Timestamp('2021-11-02 00:00:00'), Timestamp('2021-11-26 00:00:00'), Timestamp('2022-07-15 00:00:00'), Timestamp('2022-07-18 00:00:00'), Timestamp('2022-08-31 00:00:00')]


In [11]:
import akshare as ak
import pandas as pd

# 获取主力连续合约数据
df = ak.futures_main_sina(symbol="RB0", start_date="20200101", end_date="20240101")
df["日期"] = pd.to_datetime(df["日期"])

# 计算相邻交易日的收盘价差
df["价差"] = df["收盘价"].diff()

# 筛选出显著跳空（例如价差绝对值超过100点）
jump_gaps = df[abs(df["价差"]) > 100]
print("换月跳空情况：")
print(jump_gaps[["日期", "收盘价", "价差"]].tail(20))

换月跳空情况：
            日期   收盘价     价差
603 2022-07-01  4265 -113.0
609 2022-07-11  4049 -147.0
610 2022-07-12  3910 -139.0
613 2022-07-15  3589 -244.0
614 2022-07-18  3780  191.0
622 2022-07-28  4058  142.0
636 2022-08-17  3993 -160.0
638 2022-08-19  3929 -112.0
644 2022-08-29  3965 -132.0
646 2022-08-31  3677 -198.0
661 2022-09-22  3765  101.0
669 2022-10-11  3774 -101.0
712 2022-12-09  3945  106.0
718 2022-12-19  3927 -122.0
775 2023-03-16  4205 -127.0
787 2023-04-03  4044 -117.0
800 2023-04-21  3810 -104.0
808 2023-05-08  3732  110.0
820 2023-05-24  3466 -122.0
957 2023-12-13  3942 -118.0


In [14]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime

def futures_main_weekly_sina(
    symbol: str = "V0",
    start_date: str = "19900101",
    end_date: str = "22220101",
) -> pd.DataFrame:
    """
    新浪财经-期货-主力连续周线数据
    基于日线数据聚合计算，或直接调用周线接口
    """
    # 1. 获取日线数据后聚合为周线（通用方法）
    daily_df = ak.futures_main_sina(symbol=symbol, start_date=start_date, end_date=end_date)
    daily_df["日期"] = pd.to_datetime(daily_df["日期"])
    daily_df.set_index("日期", inplace=True)
    
    # 聚合逻辑：每周一至周五为一周，取首开、最高、最低、尾收、成交量和持仓量总和
    weekly_df = daily_df.resample("W-FRI").agg({
        "开盘价": "first",
        "最高价": "max",
        "最低价": "min",
        "收盘价": "last",
        "成交量": "sum",
        "持仓量": "sum",
        "动态结算价": "last"
    }).dropna()
    
    # 2. 日期筛选
    weekly_df = weekly_df[(weekly_df.index >= pd.to_datetime(start_date)) & 
                         (weekly_df.index <= pd.to_datetime(end_date))]
    weekly_df.reset_index(inplace=True)
    weekly_df.rename(columns={"index": "日期"}, inplace=True)
    return weekly_df

# 示例调用
weekly_data = futures_main_weekly_sina(symbol="V0", start_date="20240101", end_date="20251231")
print(weekly_data)

           日期     开盘价     最高价     最低价     收盘价      成交量      持仓量   动态结算价
0  2024-01-05  5880.0  5953.0  5814.0  5824.0  2755934  3516450  5843.0
1  2024-01-12  5840.0  5867.0  5754.0  5827.0  3376627  4592776  5825.0
2  2024-01-19  5835.0  5904.0  5803.0  5872.0  3078001  4140989  5856.0
3  2024-01-26  5880.0  5970.0  5823.0  5893.0  3442483  3762404  5913.0
4  2024-02-02  5898.0  5954.0  5783.0  5812.0  3252193  3855382  5818.0
..        ...     ...     ...     ...     ...      ...      ...     ...
89 2025-09-26  4955.0  4989.0  4868.0  4897.0  3940768  5599153  4902.0
90 2025-10-03  4894.0  4918.0  4838.0  4839.0  1246327  2147720  4858.0
91 2025-10-10  4825.0  4825.0  4719.0  4735.0  1396656  2361602  4754.0
92 2025-10-17  4730.0  4736.0  4644.0  4688.0  3456442  6243173  4706.0
93 2025-10-24  4696.0  4746.0  4670.0  4719.0  1801184  3593067  4706.0

[94 rows x 8 columns]


In [7]:
# 1. 提取品种代码（如"RB0"的核心代码是"RB"）
base_symbol = symbol[:-1]  # 截取前两位，"RB0"→"RB"

# 2. 确定时间范围（主力连续数据的起始年份到结束年份）
start_year = main_df.index.min().year
end_year = main_df.index.max().year

# 3. 生成该时间范围内的所有可能合约（每年1-12月）
contracts = []
for year in range(start_year, end_year + 1):
    year_code = str(year)[-2:]  # 年份后两位（2023→23）
    for month in range(1, 13):
        month_code = f"{month:02d}"  # 月份补0（1→01）
        contract = f"{base_symbol}{year_code}{month_code}"
        contracts.append(contract)
print(f"生成可能的次合约列表（共{len(contracts)}个）：{contracts[:5]}...")

生成可能的次合约列表（共72个）：['RB1801', 'RB1802', 'RB1803', 'RB1804', 'RB1805']...


In [3]:
print(y2601_df.head())
print(y2601_df.tail())

           日期   开盘价   最高价   最低价   收盘价  成交量  持仓量  动态结算价
0  2025-01-16  7732  7762  7594  7610   88   75   7672
1  2025-01-17  7580  7634  7490  7622  125  137   7550
2  2025-01-20  7604  7712  7600  7712   71  179   7640
3  2025-01-21  7690  7762  7610  7728   90  198   7688
4  2025-01-22  7740  7740  7676  7688   33  198   7702
             日期   开盘价   最高价   最低价   收盘价     成交量     持仓量  动态结算价
177  2025-10-15  8224  8282  8210  8252  206030  497389   8244
178  2025-10-16  8250  8280  8232  8256  206748  498180   8252
179  2025-10-17  8276  8320  8248  8256  295513  499960   8286
180  2025-10-20  8272  8344  8272  8298  257692  499076   8312
181  2025-10-21  8298  8348  8278  8294  249004  506908   8320


### 验证主力合约是否存在跳空问题

In [18]:
jm0_df = ak.futures_main_sina(symbol='jm0', start_date= '20250101', end_date='20251231')
print(jm0_df.head(30))
print(jm0_df.tail())

            日期     开盘价     最高价     最低价     收盘价     成交量     持仓量   动态结算价
0   2025-01-02  1162.5  1187.5  1154.0  1174.0  174740  259254  1174.0
1   2025-01-03  1174.5  1176.5  1140.0  1148.5  212419  268507  1152.5
2   2025-01-06  1144.5  1170.0  1131.5  1156.0  206940  263786  1153.5
3   2025-01-07  1162.0  1169.5  1123.0  1128.0  210354  283622  1143.5
4   2025-01-08  1132.0  1135.0  1100.0  1103.0  227315  306145  1113.5
5   2025-01-09  1103.0  1106.0  1086.0  1097.0  205515  313115  1096.5
6   2025-01-10  1096.0  1101.5  1081.5  1083.5  166844  318354  1091.0
7   2025-01-13  1090.0  1120.5  1082.0  1119.5  269632  321245  1107.0
8   2025-01-14  1119.0  1131.5  1110.5  1129.5  176569  303727  1121.0
9   2025-01-15  1127.0  1129.5  1113.0  1127.0  132833  305418  1121.5
10  2025-01-16  1127.0  1167.0  1125.5  1164.5  327022  309078  1148.0
11  2025-01-17  1160.0  1178.0  1160.0  1175.5  237866  285029  1169.0
12  2025-01-20  1170.0  1173.0  1132.0  1139.0  246664  307311  1150.5
13  20

In [15]:
daily_df = ak.futures_zh_daily_sina(symbol='jm2601')
print(daily_df.head())
print(daily_df.tail())

         date    open    high     low   close  volume  hold  settle
0  2025-01-16  1199.0  1260.0  1150.0  1258.5     172    74  1213.0
1  2025-01-17  1258.0  1271.0  1252.0  1269.0      42    97  1262.5
2  2025-01-20  1265.0  1268.5  1240.0  1246.0      65   131  1253.5
3  2025-01-21  1250.0  1260.5  1243.0  1252.5      36   133  1253.5
4  2025-01-22  1242.0  1243.5  1236.0  1240.5      33   151  1241.0
           date    open    high     low   close   volume    hold  settle
178  2025-10-16  1155.0  1186.5  1142.5  1185.5   981288  623751  1162.0
179  2025-10-17  1190.5  1194.5  1172.0  1179.0   869924  606535  1184.5
180  2025-10-20  1190.0  1238.5  1187.0  1216.0  1072864  618806  1219.5
181  2025-10-21  1216.0  1216.0  1173.0  1177.0   914637  595972  1192.5
182  2025-10-22  1180.0  1212.5  1180.0  1209.5   765910  597678  1197.0


In [8]:
print(ak.__version__)

1.17.63
